In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/insideout.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

# list of docs
# for doc in list of docs | prompt | llm
# for response in list llms response | put them all together
# final doc | prompt | llm

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context":doc.page_content, "question": question}
        ).content
        for doc in documents
    ) 

map_chain = {
    "documents": retriver,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = { "context": map_chain, "question": RunnablePassthrough() } | final_prompt | llm

chain.invoke("Describe BING BONG")


/var/folders/gh/6gsh3jws6nx9x0d7wq7516p40000gn/T/ipykernel_14428/2457118130.py:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(
/var/folders/gh/6gsh3jws6nx9x0d7wq7516p40000gn/T/ipykernel_14428/2457118130.py:22: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the langchain-unstructured package and should be used instead. To use it run `pip install -U langchain-unstructured` and import as `from langchain_unstructured import UnstructuredLoader`.
  loader = UnstructuredFileLoader("./files/insideout.txt")


AIMessage(content='Bing Bong is a pink, elephant-like creature with a cat-like face and a long, fluffy tail. He is joyful, playful, and imaginative, with a childlike innocence. Bing Bong is known for his loyalty to Riley and his selfless nature, as he ultimately sacrifices himself to help her.', response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 390, 'total_tokens': 453}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b0d7ce17-55ec-45af-ba7a-c5fe5a5c566c-0')